In [ ]:
import collections
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import requests
import torch

import preprocess
import postprocess

plt.style.use('https://raw.githubusercontent.com/RobGeada/stylelibs/main/material.mplstyle')

%load_ext autoreload
%autoreload 2

# Arguments

In [ ]:
coco_path = "/home/coco/" # if running in a docker image, this should be "/home/coco/"
image_path = coco_path+"000000339995.jpg"
model_url = "example.com" # put ODH model url here

# Preprocess
1) Load image from file
2) Pad it to square 
3) Convert into the kserve payload schema

In [ ]:
im = Image.open(image_path)
torch_im = torch.tensor(preprocess.get_as_numpy(im))
request_body = preprocess.create_kserve_payload(im)

# Send To Model
(this can take a while)

In [ ]:
response = requests.post(model_url+"/infer", json=request_body).json()

# Postprocess

In [ ]:
# Process response to PyTorch tensors
pred_dict, proto_dict = response["outputs"]
pred = torch.tensor(pred_dict['data']).reshape(pred_dict['shape'])
proto = torch.tensor(proto_dict['data']).reshape(proto_dict['shape'])

# Set up mask generator
args = {"conf":.25, "iou":.45, "agnostic_nms": False, "max_det":1000}
segPredictor = postprocess.SegmentationPredictor(overrides=args)

# Put class names into mask generator
ModelSubstitute = collections.namedtuple("ModelSubstitute", ["names"])
segPredictor.model = ModelSubstitute(names=postprocess.names)
segPredictor.batch = [["."]]

# generate masks
results = segPredictor.postprocess([pred, proto], torch_im, torch_im)

# Visualize

In [ ]:
plt.imshow(results[0].plot())
plt.axis("off")
plt.show()